## REDES NEURONALES

# Librerías

In [ ]:
import pandas as pd
import pydot
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Código

# Clasificación de la calidad del vino

In [ ]:
wine = pd.read_csv('csv/WineQT.csv')

In [ ]:
wine.head(5)

In [ ]:
wine.tail(5)

In [ ]:
wine.describe()

In [ ]:
sns.set()
fig, ax = plt.subplots(figsize=(25,11))    
sns.heatmap(wine.corr(), square=True, annot=True, linewidths=2)

In [ ]:
wine = wine.drop('Id', axis=1)
wine = wine.drop('pH', axis=1)
wine = wine.drop('chlorides', axis=1)
wine = wine.drop('free sulfur dioxide', axis=1)
wine = wine.drop('residual sugar', axis=1)

In [ ]:
train_wine, test_wine = train_test_split(wine, test_size=0.2, random_state=42)

In [ ]:
X_train = train_wine.drop(['quality'], axis=1)
y_train = train_wine['quality']
X_test = test_wine.drop(['quality'], axis=1)
y_test = test_wine['quality']

In [ ]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
def create_model(units=64, dropout_rate=0.2, optimizer='adam', learning_rate=0.001):
    model = Sequential([
        Dense(units=units, activation='relu', input_shape=(7,)),
        Dense(units=units, activation='relu'),
        Dense(units=10, activation='softmax')
    ])
    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# define hyperparameters to search
units = [16, 32, 64, 128]
dropout_rate = [0.1, 0.2, 0.3]
#optimizer = ['adam', 'rmsprop']
optimizer = ['adam', 'rmsprop', 'sgd', 'adagrad', 'adadelta', 'adamax', 'nadam']

# initialize best accuracy and corresponding model
best_accuracy = 0
best_model = None

# iterate over hyperparameters and train models
for u in units:
    for d in dropout_rate:
        for o in optimizer:
            print(f'Training model with units={u}, dropout={d}, optimizer={o}')
            
            # create and compile model
            model = create_model(units=u, dropout_rate=d, optimizer=o)
            
            # define early stopping and model checkpoint callbacks
            early_stop = EarlyStopping(monitor='val_loss', patience=5)
            checkpoint = ModelCheckpoint('best_model_wine.h5', save_best_only=True)
            
            # train model
            history = model.fit(X_train, y_train, batch_size=32, epochs=50, 
                                validation_data=(X_test, y_test), callbacks=[early_stop, checkpoint])
            
            # evaluate model on test set
            y_prob = model.predict(X_test)
            y_pred = y_prob.argmax(axis=-1)
            accuracy = accuracy_score(y_test, y_pred)
            print(f'Test accuracy: {accuracy*100}%')
            
            # save best model if accuracy improves
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_model = model

print(f'Best test accuracy: {best_accuracy*100}%')
best_model.save('final_model_wine.h5')

In [ ]:
from keras.models import load_model
from sklearn.metrics import accuracy_score
import numpy as np

# load the saved model from file
model = load_model('best_model_wine.h5')
model2 = load_model('final_model_wine.h5')

# evaluate the model on test data and get accuracy score
accuracy = model.evaluate(X_test, y_test)
accuracy2 = model2.evaluate(X_test, y_test)
print('Best Model Accuracy: %.2f%%' % (accuracy[1] * 100))
print('Final Model Accuracy: %.2f%%' % (accuracy2[1] * 100))

# Clasificar entre presencia de enfermedad del corazón o no

In [ ]:
heart = pd.read_csv('csv/heart.csv')

In [ ]:
heart.head(5)

In [ ]:
heart.tail(5)

In [ ]:
sns.set()
fig, ax = plt.subplots(figsize=(25,11))    
sns.heatmap(heart.corr(), square=True, annot=True, linewidths=2)

In [ ]:
heart = heart.apply(lambda col: pd.factorize(col, sort=True)[0])

In [ ]:
heart

In [ ]:
train_heart, test_heart = train_test_split(heart, test_size=0.2, random_state=42)

In [ ]:
X_train = train_heart.drop(['HeartDisease'], axis=1)
y_train = train_heart['HeartDisease']
X_test = test_heart.drop(['HeartDisease'], axis=1)
y_test = test_heart['HeartDisease']

In [ ]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
def create_model(units=64, dropout_rate=0.2, optimizer='adam', learning_rate=0.001):
    model = Sequential([
        Dense(units=units, activation='relu', input_shape=(7,)),
        Dense(units=units, activation='relu'),
        Dense(units=10, activation='softmax')
    ])
    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# define hyperparameters to search
units = [16, 32, 64, 128]
dropout_rate = [0.1, 0.2, 0.3]
#optimizer = ['adam', 'rmsprop']
optimizer = ['adam', 'rmsprop', 'sgd', 'adagrad', 'adadelta', 'adamax', 'nadam']

# initialize best accuracy and corresponding model
best_accuracy = 0
best_model = None

# iterate over hyperparameters and train models
for u in units:
    for d in dropout_rate:
        for o in optimizer:
            print(f'Training model with units={u}, dropout={d}, optimizer={o}')
            
            # create and compile model
            model = create_model(units=u, dropout_rate=d, optimizer=o)
            
            # define early stopping and model checkpoint callbacks
            early_stop = EarlyStopping(monitor='val_loss', patience=5)
            checkpoint = ModelCheckpoint('best_model_heart.h5', save_best_only=True)
            
            # train model
            history = model.fit(X_train, y_train, batch_size=32, epochs=50, 
                                validation_data=(X_test, y_test), callbacks=[early_stop, checkpoint])
            
            # evaluate model on test set
            y_prob = model.predict(X_test)
            y_pred = y_prob.argmax(axis=-1)
            accuracy = accuracy_score(y_test, y_pred)
            print(f'Test accuracy: {accuracy*100}%')
            
            # save best model if accuracy improves
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_model = model

print(f'Best test accuracy: {best_accuracy*100}%')
best_model.save('final_model_heart.h5')

In [ ]:
from keras.models import load_model
from sklearn.metrics import accuracy_score
import numpy as np

# load the saved model from file
model = load_model('best_model_heart.h5')
model2 = load_model('final_model_heart.h5')

# evaluate the model on test data and get accuracy score
accuracy = model.evaluate(X_test, y_test)
accuracy2 = model2.evaluate(X_test, y_test)
print('Best Model Accuracy: %.2f%%' % (accuracy[1] * 100))
print('Final Model Accuracy: %.2f%%' % (accuracy2[1] * 100))